In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
IMPORTANT_LMS = [
"NOSE",
"LEFT_SHOULDER",
"RIGHT_SHOULDER",
"LEFT_HIP",
"RIGHT_HIP",
"LEFT_KNEE",
"RIGHT_KNEE",
"LEFT_ANKLE",
"RIGHT_ANKLE",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [5]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [6]:
# load dataset
df = pd.read_csv("./train.csv")

# Categorizing label
df.loc[df["label"] == "no_error", "label"] = 0
df.loc[df["label"] == "knees_inward_error", "label"] = 1

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 8660 
Number of columns: 37

Labels: 
0    6935
1    1725
Name: label, dtype: int64



In [7]:
# Standard Scaling of features
# Dump input scaler
with open("./model/ki_input_scaler.pkl", "rb") as f2:
    sc = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [9]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [10]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")
            

def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [11]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [12]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)
tuner_3l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 06s]
accuracy: 0.8415126800537109

Best accuracy So Far: 0.929416835308075
Total elapsed time: 00h 01m 28s
INFO:tensorflow:Oracle triggered exit


In [13]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-2: 480 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-3: 2 units, func:  <function softmax at 0x000002B2E3C4CE50>

Other params:
learning_rate: 0.001
Epoch 1/100
693/693 [==============================] - 2s 3ms/step - loss: 0.4014 - accuracy: 0.8279 - val_loss: 0.3680 - val_accuracy: 0.8487
Epoch 2/100
693/693 [==============================] - 2s 2ms/step - loss: 0.3405 - accuracy: 0.8587 - val_loss: 0.3293 - val_accuracy: 0.8637
Epoch 3/100
693/693 [==============================] - 2s 3ms/step - loss: 0.2977 - accuracy: 0.8834 - val_loss: 0.2860 - val_accuracy: 0.8776
Epoch 4/100
693/693 [==============================] - 2s 2ms/step - loss: 0.2667 - accuracy: 0.8951 - val_loss: 0.2922 - val_accuracy: 0.8782
Epoch 5/100
693/693 [==============================] - 2s 2ms/step - loss: 0.2482 - accuracy: 0.9052 - val_loss: 0.2760 - val_accuracy: 0.8857
Epoch 6/100
693/693 [====

In [14]:
final_models["3_layers"] = model_3l

In [15]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [16]:
import keras
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_1'
)

tuner_5l.search(
    x_train, 
    y_train, 
    epochs=10, 
    callbacks=[keras.callbacks.EarlyStopping(patience=5)]
)

Trial 30 Complete [00h 00m 08s]
accuracy: 0.8589780330657959

Best accuracy So Far: 0.9590069055557251
Total elapsed time: 00h 02m 26s
INFO:tensorflow:Oracle triggered exit


In [17]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-2: 160 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-3: 480 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-4: 288 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-5: 2 units, func:  <function softmax at 0x000002B2E3C4CE50>

Other params:
learning_rate: 0.001
Epoch 1/100
693/693 [==============================] - 4s 4ms/step - loss: 0.4117 - accuracy: 0.8246 - val_loss: 0.3732 - val_accuracy: 0.8401
Epoch 2/100
693/693 [==============================] - 3s 4ms/step - loss: 0.3294 - accuracy: 0.8682 - val_loss: 0.3081 - val_accuracy: 0.8799
Epoch 3/100
693/693 [==============================] - 3s 4ms/step - loss: 0.2845 - accuracy: 0.8893 - val_loss: 0.2737 - val_accuracy: 0.8834
Epoch 4/100
693/693 [==============================] - 3s 4ms/step - loss: 0.2481 - accuracy: 0.9030 - val_loss: 0.2658 - val_accuracy: 0.9001
Epoch 5/100
693/693 [=================

In [18]:
final_models["5_layers"] = model_5l

In [19]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [20]:
def get_early_stopping_callback():
    return keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3
    )

tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_4'
)
tuner_7lD.search(x_train, y_train, epochs=10, callbacks=[get_early_stopping_callback()])

Trial 30 Complete [00h 00m 13s]
accuracy: 0.851760983467102

Best accuracy So Far: 0.9461604952812195
Total elapsed time: 00h 03m 06s
INFO:tensorflow:Oracle triggered exit


In [21]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-2: 160 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-3: 0 units, func: None
Layer-4: 352 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-5: 0 units, func: None
Layer-6: 256 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-7: 2 units, func:  <function softmax at 0x000002B2E3C4CE50>

Other params:
dropout_1: 0.1
dropout_2: 0.1
learning_rate: 0.001
Epoch 1/100
693/693 [==============================] - 4s 4ms/step - loss: 0.4156 - accuracy: 0.8233 - val_loss: 0.3726 - val_accuracy: 0.8406
Epoch 2/100
693/693 [==============================] - 3s 5ms/step - loss: 0.3456 - accuracy: 0.8597 - val_loss: 0.3326 - val_accuracy: 0.8643
Epoch 3/100
693/693 [==============================] - 4s 5ms/step - loss: 0.3088 - accuracy: 0.8795 - val_loss: 0.2884 - val_accuracy: 0.8886
Epoch 4/100
693/693 [==============================] - 3s 4ms/step - loss: 0.2796 - accuracy:

In [22]:
final_models["7_layers_with_dropout"] = model_7lD

In [23]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [24]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_5'
)
tuner_7l.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Trial 30 Complete [00h 00m 19s]
accuracy: 0.9577078819274902

Best accuracy So Far: 0.9603059887886047
Total elapsed time: 00h 02m 59s
INFO:tensorflow:Oracle triggered exit


In [25]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-2: 224 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-3: 320 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-4: 320 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-5: 448 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-6: 288 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-7: 2 units, func:  <function softmax at 0x000002B2E3C4CE50>

Other params:
learning_rate: 0.001
Epoch 1/100
693/693 [==============================] - 5s 7ms/step - loss: 0.4187 - accuracy: 0.8252 - val_loss: 0.3637 - val_accuracy: 0.8447
Epoch 2/100
693/693 [==============================] - 5s 7ms/step - loss: 0.3377 - accuracy: 0.8616 - val_loss: 0.3180 - val_accuracy: 0.8695
Epoch 3/100
693/693 [==============================] - 4s 6ms/step - loss: 0.2984 - accuracy: 0.8803 - val_loss: 0.3135 - val_accuracy: 0.8712
Epoch 4/100
693/693 [==============================

In [26]:
final_models["7_layers"] = model_7l

In [27]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-2: 480 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-3: 2 units, func:  <function softmax at 0x000002B2E3C4CE50>

5_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-2: 160 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-3: 480 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-4: 288 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-5: 2 units, func:  <function softmax at 0x000002B2E3C4CE50>

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-2: 160 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-3: 0 units, func: None
Layer-4: 352 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-5: 0 units, func: None
Layer-6: 256 units, func:  <function relu at 0x000002B2E3C548B0>
Layer-7: 2 units, func:  <funct

In [28]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.975, 0.879, 0.0]","[0.97, 0.897, 0.0]","[0.972, 0.888, 0.0]","[[1351, 42, 0], [35, 304, 0], [0, 0, 0]]"
1,7_layers,"[0.976, 0.86, 0.0]","[0.964, 0.903, 0.0]","[0.97, 0.881, 0.0]","[[1343, 50, 0], [33, 306, 0], [0, 0, 0]]"
2,7_layers_with_dropout,"[0.961, 0.896, 0.0]","[0.976, 0.838, 0.0]","[0.969, 0.866, 0.0]","[[1360, 33, 0], [55, 284, 0], [0, 0, 0]]"
3,5_layers,"[0.969, 0.827, 0.0]","[0.955, 0.873, 0.0]","[0.962, 0.849, 0.0]","[[1331, 62, 0], [43, 296, 0], [0, 0, 0]]"


In [29]:
# load dataset
test_df = pd.read_csv("./train.csv")

# Categorizing label
test_df.loc[test_df["label"] == "no_error", "label"] = 0
test_df.loc[test_df["label"] == "knees_inward_error", "label"] = 1

print(f'Number of rows: {test_df.shape[0]} \nNumber of columns: {test_df.shape[1]}\n')
print(f"Labels: \n{test_df['label'].value_counts()}\n")

Number of rows: 8660 
Number of columns: 37

Labels: 
0    6935
1    1725
Name: label, dtype: int64



In [30]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [31]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.993, 0.961, 0.0]","[0.99, 0.972, 0.0]","[0.992, 0.966, 0.0]","[[6867, 68, 0], [49, 1676, 0], [0, 0, 0]]"
1,7_layers,"[0.992, 0.932, 0.0]","[0.983, 0.968, 0.0]","[0.987, 0.95, 0.0]","[[6814, 121, 0], [55, 1670, 0], [0, 0, 0]]"
2,7_layers_with_dropout,"[0.983, 0.959, 0.0]","[0.99, 0.93, 0.0]","[0.986, 0.945, 0.0]","[[6867, 68, 0], [120, 1605, 0], [0, 0, 0]]"
3,5_layers,"[0.986, 0.898, 0.0]","[0.973, 0.945, 0.0]","[0.98, 0.921, 0.0]","[[6750, 185, 0], [95, 1630, 0], [0, 0, 0]]"


In [32]:
final_models["3_layers"].save("./model/ki_dp.h5")

In [33]:
for model_name, model in final_models.items():
    model.save(f"./model/{model_name}.h5")